In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

### 解析数据

In [2]:
data = pd.read_csv('train.csv')
data

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
fig_datas = data.drop('label', 1) / 255
label_datas = data['label']

### 可视化

In [4]:
# transform a n*n size series to a n*n data frame 
def transformDataFrameFromSeries(data):
    df = pd.DataFrame()
    size = int(math.sqrt(data.size))
    for i in range(size):
        df = df.append(data.iloc[i * size : (i * size + size)].reset_index(drop=True),ignore_index=False)
    return df

In [5]:
# show some fig
plt.figure(figsize=(18,9))
for i in range(25):
    ax = plt.subplot2grid((5,5),(i / 5, i % 5))
    fig_data = fig_datas.iloc[i]
    ax.imshow(transformDataFrameFromSeries(fig_data), cmap=plt.cm.gray_r)
    plt.title('fig %s label:%s' % (i, label_datas[i]))
plt.tight_layout()
plt.show()

In [107]:
# save some fig
for i in range(500):
    fig_data = fig_datas.iloc[i]
    fig, ax = plt.subplots()
    ax.imshow(transformDataFrameFromSeries(fig_data), cmap=plt.cm.gray_r)
    plt.savefig('./figs/' + str(i) + '-' + str(label_datas[i]) + '.png')

可以看到不同图的有效区域有差异，同时有些图还有旋转的效果。

有没有必要对数据作进一步降维处理？

直接上random forest试试效果吧

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)

train_x = data.drop(['label'], axis=1)
train_y = data['label']

rf.fit(train_x, train_y)

In [7]:
test_data = pd.read_csv('test.csv')

In [8]:
test_data['label'] = rf.predict(test_data)
test_fig_datas = test_data.drop('label', 1) / 255
test_label_datas = test_data['label']

In [15]:
# show some fig
plt.figure(figsize=(18,9))
for i in range(25):
    ax = plt.subplot2grid((5,5),(i / 5, i % 5))
    fig_data = test_fig_datas.iloc[i]
    ax.imshow(transformDataFrameFromSeries(fig_data), cmap=plt.cm.gray_r)
    plt.title('fig %s label:%s' % (i, test_label_datas[i]))
plt.tight_layout()
plt.show()

In [14]:
submission = pd.DataFrame({
			"ImageId": range(1, test_data.shape[0]+1),
			"label": test_data['label']})
submission.to_csv("random_forest.csv", index=False)

什么参数都没调，数据也没做处理，分数已经达到0.96600。

### 优化尝试

从数据清洗上看，首先一份向量由784个像素点组成，每个像素点取值范围是0-255，这个样本空间就很大了，而其实每个像素点应该只有两个有效状态，是不是可以尝试对数据进行进一步降维处理。